# **Tutorial: Random Forests**
### By Kostas Hatalis

___
## Bagging

Bagging, also known as bootstrap aggregation, is an ensemble method involving training the same algorithm many times using different subsets sampled with replacement (known as boostrap sampling) from the training data. It then aggregates their individual predictions (either by voting or by averaging) to form a final prediction. Such a meta-estimator can typically be used as a way to **reduce the variance** of a base estimator (e.g., a CART or ANN), by introducing randomization into its construction procedure and then making an ensemble out of it.



___
## Random Forests


A random forest is a meta estimator that fits a number of decision tree classifiers on various sub-samples of the dataset and uses averaging to improve the predictive accuracy and control over-fitting. The sub-sample size is always the same as the original input sample size but the samples are drawn with replacement if bootstrap=True (default).

### Code Example

### Random Forests: Hyperparameters
